In [1]:
import pandas as pd
from pycocotools.coco import COCO

dataF = 'G:/edu/sem2/766/proj/ML_OD_Benchmarking/data/'
metricF = dataF+'outputs/metrics/'
anntF = dataF+'inputs/annotations/instances_val2017.json'
coco=COCO(anntF)



loading annotations into memory...
Done (t=0.70s)
creating index...
index created!


In [2]:
transformNames = [
 'None',
 'gaussianblur_1',
 'gaussianblur_10',
 'gaussianblur_20',
 'superpixels_0p1',
 'superpixels_0p5',
 'superpixels_0p85',
 'colorspace_25',
 'colorspace_50',
 'averageblur_5_11',
 'medianblur_1',
 'sharpen_0',
 'sharpen_1',
 'sharpen_2',
 'addintensity_-80',
 'addintensity_80',
 'elementrandomintensity_1',
 'multiplyintensity_0p25',
 'multiplyintensity_2',
 'contrastnormalization_0',
 'contrastnormalization_1',
 'contrastnormalization_2',
 'elastic_1'
 'scaled_1p25',
 'scaled_0p75',
 'scaled_0p5',
 'scaled_(1p25, 1p0)',
 'scaled_(0p75, 1p0)',
 'scaled_(1p0, 1p25)',
 'scaled_(1p0, 0p75)',
 'translate_(0p1, 0p1)',
 'translate_(0p1, -0p1)',
 'translate_(-0p1, 0p1)',
 'translate_(-0p1, -0p1)',
 'translate_(0p1, 0)',
 'translate_(-0p1, 0)',
 'translate_(0, 0p1)',
 'translate_(0, -0p1)',
 'rotated_3',
 'rotated_5',
 'rotated_10',
 'rotated_45',
 'rotated_60',
 'rotated_90',
 'flipH',
 'flipV',
 'dropout',
 ]
modelNames = ['e2e_faster_rcnn_R-50-C4_2x',
'e2e_faster_rcnn_R-50-FPN_2x',
'e2e_faster_rcnn_R-101-FPN_2x',
'e2e_faster_rcnn_X-101-64x4d-FPN_2x',
'e2e_mask_rcnn_R-50-C4_2x',
'e2e_mask_rcnn_R-50-FPN_2x',
'e2e_mask_rcnn_R-101-FPN_2x',
'retinanet_R-50-FPN_2x',
'retinanet_R-101-FPN_2x',
'retinanet_X-101-64x4d-FPN_2x']
cats = coco.loadCats(coco.getCatIds())


In [3]:
args = ['',modelNames[0], 'None', 'all']

if args[1] not in modelNames:
    raise ValueError('Model: '+args[1]+' not found')
model = args[1]#Single model only, not all
if args[2]=='all':
    transform = transformNames
elif args[2] not in transformNames:
    raise ValueError('Transform: '+args[2]+' not found')
else:
    transform = [args[2]]
if args[3] =='all':
    classes = [cat['id'] for cat in cats]
else:
    try:
        id = int(args[3])
    except:
        cat = [cat for cat in cats if (cat['name'] == args[3] or cat['supercategory']==args[3])]
        if len(cat)==0:
            raise ValueError('Category name: '+args[3]+' not found')
        else:
            id = cat[0]['id']
    if id<1 or id>80:
        raise ValueError('Category ID: '+str(id)+' not found')
    classes = [id]


In [4]:
def partition(model,transform,classes):
    file = metricF+transform+'_'+model+'.csv'
    dt = pd.read_csv(file)
    dt = dt[dt['actual'].isin(classes) | dt['predicted'].isin(classes)]#Take both in order to get TP,FN,FN
    return dt
d1 = partition(model, transformNames[0], [1])
d2 = partition(model, transformNames[1], [1])
d3 = pd.concat([d1,d2], ignore_index=True)
print(len(d1))
print(len(d2))
print(len(d3))

23599
7218
30817


In [5]:
def multPartition(model, transforms, classes):
    prt = []
    for tr in transforms:
        prt.append(partition(model, tr, classes))
    return pd.concat(prt)
print(len(multPartition(model,transformNames[0:2],classes)))

84121


In [6]:
def TpFpFn(dt,Class):
    TP = dt[(dt.actual==Class) & (dt.predicted==Class)]
    FP = dt[(dt.actual!=Class) & (dt.predicted==Class)]
    FN = dt[(dt.actual==Class) & (dt.predicted!=Class)]
    TN = dt[(dt.actual!=Class) & (dt.predicted!=Class)]#Zero if partition is used
    return {'tp':sum(TP.wt),'fp':sum(FP.wt),'fn':sum(FN.wt),'tn':sum(TN.wt)}
dt = partition(model, 'None', [1])
met = TpFpFn(dt,1)
print(met)

{'tp': 3009.7311632790147, 'fp': 1326.064786353547, 'fn': 7994.268836720916, 'tn': 0}


In [7]:
metrics = {
    'precision':met['tp']/(met['tp']+met['fp']),
    'recall':met['tp']/(met['tp']+met['fn']),
    'accuracy':(met['tp']+met['tn'])/sum(met.values()),
    'TP':met['tp'],
    'FP':met['fp'],
    'FN':met['fn'],
    'TN':met['tn']
}
print(metrics)

{'precision': 0.6941588576219956, 'recall': 0.273512464856328, 'accuracy': 0.2440969464012946, 'TP': 3009.7311632790147, 'FP': 1326.064786353547, 'FN': 7994.268836720916, 'TN': 0}


In [16]:
from sklearn.metrics import confusion_matrix 
import numpy as np
confusion_matrix(
    list(dt.actual), 
    list(dt.predicted),
    labels=[np.nan]+[cat['id'] for cat in cats],
    sample_weight = list(dt.wt))

ValueError: continuous is not supported

In [25]:
labels=[cat['id'] for cat in cats]
dt = multPartition(model,['None'],labels)
matrix = [[sum(dt[(dt.actual==actual) & (dt.predicted==predicted)].wt) for actual in labels] for predicted in labels]

In [31]:
mat=pd.DataFrame(np.matrix(matrix))
mat.columns = ['A_'+str(i) for i in labels]
mat.index = ['P_'+str(i) for i in labels]
mat

,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,A_9,A_10,...,A_80,A_81,A_82,A_84,A_85,A_86,A_87,A_88,A_89,A_90
P_1,3009.731163,11.516667,49.208001,38.038894,3.711905,51.384145,28.682107,24.525000,16.061706,0.583333,...,0.000000,1.000000,3.944444,3.177356,0.5,2.20,0.000000,5.283333,1.0,0.0
P_2,7.107152,49.559314,0.665404,5.500000,0.000000,0.944444,0.166667,0.500000,0.400000,0.000000,...,0.000000,0.000000,0.000000,0.500000,0.0,0.00,0.000000,0.000000,0.0,0.0
P_3,46.842346,1.709804,233.765708,6.974892,1.202381,8.658333,1.583333,26.822222,4.333333,0.166667,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.500000,0.0,0.0
P_4,7.241255,3.633333,4.404401,91.966830,0.000000,0.366667,0.111111,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0
P_5,0.047619,0.000000,0.500000,0.000000,82.060714,0.166667,1.000000,0.166667,1.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0
P_6,6.143689,0.000000,1.972727,0.000000,0.333333,85.630862,3.861111,5.305556,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0
P_7,1.274457,0.333333,0.250000,0.000000,0.000000,4.126623,108.262338,2.333333,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.0,0.00,0.000000,0.000000,0.0,0.0
P_8,9.892553,0.125000,14.448802,0.985714,0.650000,7.010714,3.750000,71.894444,0.666667,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0
P_9,4.538462,0.705882,0.305556,0.500000,0.200000,0.333333,0.333333,1.200000,70.404960,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.100000,0.0,0.0
P_10,8.735556,0.000000,14.292749,0.833333,0.000000,6.444444,5.000000,4.833333,0.500000,47.083333,...,0.000000,0.000000,0.000000,0.333333,0.0,0.00,0.000000,0.250000,0.0,0.0
